<center>
    This is Set 5: Column Aggregations (Exercises 41-50)
</center>

**Prerequisites**
* The sample dataset [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) from sklearn will be used for the exercises.

In [3]:
# !pip3 install scikit-learn==1.0.2


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/26 09:07:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
import pandas as pd
from sklearn.datasets import load_wine

data = spark.createDataFrame(load_wine(as_frame=True).frame)
data.show()


+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------+
|alcohol|malic_acid| ash|alcalinity_of_ash|magnesium|total_phenols|flavanoids|nonflavanoid_phenols|proanthocyanins|color_intensity| hue|od280/od315_of_diluted_wines|proline|target|
+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+------+
|  14.23|      1.71|2.43|             15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                        3.92| 1065.0|     0|
|   13.2|      1.78|2.14|             11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|                         3.4| 1050.0|     0|
|  13.16|      2.36|2.67|             18.6|    101.0|          2.8|      3.24|                 

**Exercise 41: Calculate the sum of column `ash` in `data`**

In [11]:
data.agg(F.sum(data["ash"])).collect()[0][0]


421.24

**Exercise 42: Calculate the average value of each column in `data`**

In [29]:
data.agg(*[F.mean(col).alias(col) for col in data.columns]).first().asDict()


{'alcohol': 13.00061797752809,
 'malic_acid': 2.3363483146067416,
 'ash': 2.3665168539325845,
 'alcalinity_of_ash': 19.49494382022472,
 'magnesium': 99.74157303370787,
 'total_phenols': 2.295112359550562,
 'flavanoids': 2.0292696629213487,
 'nonflavanoid_phenols': 0.36185393258426973,
 'proanthocyanins': 1.590898876404494,
 'color_intensity': 5.058089882022472,
 'hue': 0.9574494382022471,
 'od280/od315_of_diluted_wines': 2.611685393258427,
 'proline': 746.8932584269663,
 'target': 0.9382022471910112}

**Exercise 43: Calculate the minimum value and maximum value of each column in `data` where `target` is 1**

In [36]:
data.where(data["target"]==1).groupBy().min().first().asDict()

{'min(alcohol)': 11.03,
 'min(malic_acid)': 0.74,
 'min(ash)': 1.36,
 'min(alcalinity_of_ash)': 10.6,
 'min(magnesium)': 70.0,
 'min(total_phenols)': 1.1,
 'min(flavanoids)': 0.57,
 'min(nonflavanoid_phenols)': 0.13,
 'min(proanthocyanins)': 0.41,
 'min(color_intensity)': 1.28,
 'min(hue)': 0.69,
 'min(od280/od315_of_diluted_wines)': 1.59,
 'min(proline)': 278.0,
 'min(target)': 1}

In [37]:
data.where(data["target"]==1).groupBy().max().first().asDict()

{'max(alcohol)': 13.86,
 'max(malic_acid)': 5.8,
 'max(ash)': 3.23,
 'max(alcalinity_of_ash)': 30.0,
 'max(magnesium)': 162.0,
 'max(total_phenols)': 3.52,
 'max(flavanoids)': 5.08,
 'max(nonflavanoid_phenols)': 0.66,
 'max(proanthocyanins)': 3.58,
 'max(color_intensity)': 6.0,
 'max(hue)': 1.71,
 'max(od280/od315_of_diluted_wines)': 3.69,
 'max(proline)': 985.0,
 'max(target)': 1}

**Exercise 44: Calculate the median value of columns `flavanoids`, `hue` and `magnesium` in `data`**

In [45]:
cols = ["flavanoids", "hue", "magnesium"]

data.approxQuantile(cols, [0.5], 0.0)

[[2.13], [0.96], [98.0]]

**Exercise 45: Calculate the mode of columns `alcohol` and `proline` in `data`**

In [49]:
[[i, data.groupby(i).count().orderBy("count", ascending=False).first()[0]] for i in ["alcohol", "proline"]]


[['alcohol', 13.05], ['proline', 680.0]]

**Exercise 46: Calculate the standard deviation, skewness and excess kurtosis of each column in `data`**

In [8]:
data.agg(["std", "skew", "kurtosis"])


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
skew,-0.051482,1.039651,-0.176699,0.213047,1.098191,0.086639,0.025344,0.450151,0.517137,0.868585,0.021091,-0.307285,0.767822,0.107431
kurtosis,-0.852500,0.299207,1.143978,0.487942,2.104991,-0.835627,-0.880382,-0.637191,0.554649,0.381522,-0.344096,-1.086435,-0.248403,-1.322787


**Exercise 47: Calculate the number of unique values in each column of `data`**

In [9]:
data.nunique(axis=0)


alcohol                         126
malic_acid                      133
ash                              79
alcalinity_of_ash                63
magnesium                        53
total_phenols                    97
flavanoids                      132
nonflavanoid_phenols             39
proanthocyanins                 101
color_intensity                 132
hue                              78
od280/od315_of_diluted_wines    122
proline                         121
target                            3
dtype: int64

**Exercise 48: Calculate the covariance between columns `alcohol` and `color_intensity` in `data`**

In [10]:
data[["alcohol", "color_intensity"]].cov().loc["alcohol", "color_intensity"]


1.0282825443106076

**Exercise 49: Calculate the correlation between columns `hue` and `color_intensity` in `data`**

In [11]:
data[["hue", "color_intensity"]].corr().loc["hue", "color_intensity"]


-0.5218131932287572

**Exercise 50: Calculate the correlation matrix between all pairs of columns in `data` and assign it to `corr_matrix`**

In [12]:
corr_matrix = data.corr().round(2)
corr_matrix


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
alcohol,1.00,0.09,0.21,-0.31,0.27,0.29,0.24,-0.16,0.14,0.55,-0.07,0.07,0.64,-0.33
malic_acid,0.09,1.00,0.16,0.29,-0.05,-0.34,-0.41,0.29,-0.22,0.25,-0.56,-0.37,-0.19,0.44
ash,0.21,0.16,1.00,0.44,0.29,0.13,0.12,0.19,0.01,0.26,-0.07,0.00,0.22,-0.05
alcalinity_of_ash,-0.31,0.29,0.44,1.00,-0.08,-0.32,-0.35,0.36,-0.20,0.02,-0.27,-0.28,-0.44,0.52
magnesium,0.27,-0.05,0.29,-0.08,1.00,0.21,0.20,-0.26,0.24,0.20,0.06,0.07,0.39,-0.21
total_phenols,0.29,-0.34,0.13,-0.32,0.21,1.00,0.86,-0.45,0.61,-0.06,0.43,0.70,0.50,-0.72
flavanoids,0.24,-0.41,0.12,-0.35,0.20,0.86,1.00,-0.54,0.65,-0.17,0.54,0.79,0.49,-0.85
nonflavanoid_phenols,-0.16,0.29,0.19,0.36,-0.26,-0.45,-0.54,1.00,-0.37,0.14,-0.26,-0.50,-0.31,0.49
proanthocyanins,0.14,-0.22,0.01,-0.20,0.24,0.61,0.65,-0.37,1.00,-0.03,0.30,0.52,0.33,-0.50
color_intensity,0.55,0.25,0.26,0.02,0.20,-0.06,-0.17,0.14,-0.03,1.00,-0.52,-0.43,0.32,0.27


✅ This completes Set 5: Column Aggregations (Exercises 41-50)  
Original exercises for Datatable package can be found [here](https://github.com/vopani/datatableton)